#Hardware#

In [ ]:
!nvidia-smi

Sat Apr 20 15:49:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import pandas as pd
embeddings = pd.read_csv("/content/section_embeddings_noillustrations_24l24h.csv")

data = pd.read_csv("/content/sectionsdataset_withoutillustrations - Sheet1.csv")

In [ ]:
# tensorembeddings = embeddings
# for i in (embeddings.columns):
#   tensorembeddings[i] = embeddings[i].str.extract(r'(\d*\.\d*)').astype(float)


In [ ]:
embeddings.shape

(506, 768)

In [ ]:
sections = data["Description"].values
sectionnames = data["Section"].values
sectionnumber = data["Number"].values

In [ ]:
sections.shape


(506,)

In [ ]:
len(embeddings)

506

In [ ]:
# Convert embeddings to numpy arrays
sentence_vectors = embeddings

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import BertTokenizer, BertModel
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
import numpy as np

# Load pre-trained BERT tokenizer and model
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',num_hidden_layers=24 , num_attention_heads=24 )

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Function to tokenize, remove stop words, and encode sentences
def encode_sentences(sentences):
    cleaned_sentences = []
    for sentence in sentences:
        # Remove stop words
        cleaned_sentence = ' '.join([word for word in sentence.split() if word.lower() not in stop_words])
        cleaned_sentences.append(cleaned_sentence)

    # Tokenize and encode the cleaned sentences
    encoded_sentences = tokenizer(cleaned_sentences, return_tensors='pt', padding=True, truncation=True)

    return encoded_sentences

# Encode the sections
encoded_sections = encode_sentences(sections)

# Get BERT embeddings for the tokenized sections
def calculate_embeddings(encoded_sentences):
    with torch.no_grad():
        outputs = model(**encoded_sentences)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling of token embeddings
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=-1)  # L2 normalize
    return embeddings

#batch processing
def batch_process_sentences(sentences, batch_size=50):
    embeddings_list = []
    num_batches = (len(sentences) + batch_size - 1) // batch_size

    for i in range(num_batches):

        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(sentences))
        batch_sentences = sentences[start_idx:end_idx]
        print("Encoding sections from ",start_idx," to ",end_idx)
        # Encode batch of sentences
        encoded_sentences = encode_sentences(batch_sentences)

        # Calculate embeddings for batch
        embeddings = calculate_embeddings(encoded_sentences)
        embeddings_list.extend(embeddings)

    # Concatenate embeddings from all batches

    return embeddings_list


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of BertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.12.attention.output.LayerNorm.bias', 'bert.encoder.layer.12.attention.output.LayerNorm.weight', 'bert.encoder.layer.12.attention.output.dense.bias', 'bert.encoder.layer.12.attention.output.dense.weight', 'bert.encoder.layer.12.attention.self.key.bias', 'bert.encoder.layer.12.attention.self.key.weight', 'bert.encoder.layer.12.attention.self.query.bias', 'bert.encoder.layer.12.attention.self.query.weight', 'bert.encoder.layer.12.attention.self.value.bias', 'bert.encoder.layer.12.attention.self.value.weight', 'bert.encoder.layer.12.intermediate.dense.bias', 'bert.encoder.layer.12.intermediate.dense.weight', 'bert.encoder.layer.12.output.LayerNorm.bias', 'bert.encoder.layer.12.output.LayerNorm.weight', 'bert.encoder.layer.12.output.d

#Finding similarity#

In [ ]:
def calculate_similarity(input_text,sentence_vectors=sentence_vectors):
    # Encode the input text
    encoded_input = encode_sentences([input_text])

    # Get BERT embeddings for the tokenized input text
    with torch.no_grad():
        input_embedding = model(**encoded_input).last_hidden_state.mean(dim=1)  # Mean pooling of token embeddings
        input_embedding = torch.nn.functional.normalize(input_embedding, p=2, dim=-1)  # L2 normalize

    similarity_scores = cosine_similarity(input_embedding, sentence_vectors)

    # Rank sentences based on similarity scores
    ranked_sentences = sorted(zip(sectionnames, similarity_scores[0]), key=lambda x: x[1], reverse=True)

    return ranked_sentences

In [ ]:
calculate_similarity("A government official misuses his position and stole public funds")[:10]

[('Coin defined ', 0.08304483199673941),
 ('Waging, or attempting to wage war, or abetting waging of war, against the Government of India',
  0.058092611180885746),
 ('Counterfeiting Government stamp ', 0.051316379677564236),
 ('Fraudulently or dishonestly diminishing weight or altering composition of coin ',
  0.05005387767714778),
 ('Person employed in mint causing coin to be of different weight or composition from that fixed by law ',
  0.04954828246758841),
 ('Counterfeiting device or mark used for authenticating documents other than those described in section 467, or possessing counterfeit marked material ',
  0.049271018106036964),
 ('Making or selling instrument for counterfeiting Government stamp ',
  0.04859598548106513),
 ('Forged document ', 0.04835798420392587),
 ('Possession of Indian coin by person who knew it to be counterfeit when he became possessed thereof ',
  0.04685408409780542),
 ('Fraudulently or dishonestly diminishing weight or altering composition of Indian co

In [ ]:
len(sections)

506

In [ ]:
sections[sectionnames=='False information, with intent to cause public servant to use his lawful power to the injury of another person']

array(['Whoever gives to any public servant any information which he knows or believes to be false, intending thereby to cause, or knowing it to be likely that he will thereby cause such public servant -to do or omit anything which such public servant ought not to do or omit if the true state of facts respecting which such information is given were known by him, orto use the lawful power of such public servant to the injury or annoyance of any person, shall be punished with imprisonment of either description for a term which may extend to six months, or with fine which may extend to one thousand rupees, or with both.'],
      dtype=object)

In [ ]:
embed = pd.DataFrame(sentence_vectors)
embed

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.006125,-0.032665,0.014649,0.026808,0.059089,-0.009365,-0.016808,-0.010852,-0.042066,0.013446,...,-0.002032,-0.064723,-0.066550,-0.009346,0.004759,0.030237,0.056819,0.062340,0.046500,0.017790
1,0.026880,-0.030728,0.010022,0.008024,0.059553,-0.041781,-0.033307,0.024806,-0.025195,0.005954,...,-0.004732,-0.088148,-0.040419,-0.013646,0.006121,0.009879,0.072345,0.033862,0.025697,0.026197
2,-0.014643,-0.044208,0.016928,0.013390,0.019684,-0.030419,-0.008244,0.029799,-0.006144,0.012375,...,-0.005040,-0.091151,-0.047512,-0.023470,0.003134,0.009436,0.067596,0.018500,0.028217,-0.014844
3,-0.009372,-0.051847,0.017720,-0.008822,0.035534,-0.017540,-0.023276,0.029102,-0.031280,0.014001,...,0.018803,-0.095583,-0.047304,-0.004684,0.016457,0.024335,0.071600,0.017466,0.041474,0.006945
4,-0.003544,-0.027803,0.003720,-0.005375,0.063969,-0.028282,-0.026335,-0.014699,-0.030639,0.020134,...,0.000567,-0.089807,-0.029630,-0.027142,0.003355,0.019631,0.069448,0.022996,0.030006,0.017287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.007725,-0.035536,0.022585,-0.008584,0.045320,-0.019671,-0.028757,0.020993,-0.016390,0.015359,...,0.005521,-0.070507,-0.047128,-0.011143,0.016921,-0.007654,0.064237,0.016454,0.037350,0.027830
502,-0.019404,-0.029230,0.012766,-0.014646,0.037214,-0.036061,-0.017597,0.015215,-0.017839,0.014814,...,0.008720,-0.097858,-0.038218,-0.008666,0.016269,-0.026597,0.055956,0.004337,0.053822,0.021270
503,-0.014019,0.002406,0.032784,-0.012924,0.054703,0.006240,-0.042927,0.008384,-0.005286,0.028453,...,-0.018657,-0.054719,-0.030794,-0.010900,0.022542,-0.024449,0.048990,0.019509,0.029780,0.017413
504,-0.020260,-0.019837,0.027415,0.002004,0.053177,-0.026605,-0.040737,0.008822,-0.004693,0.043284,...,0.002584,-0.057660,-0.039622,-0.013349,0.017212,-0.038237,0.041818,0.022263,0.026249,0.025970
